# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

# Hackathon 2: Voice commands based food ordering system

### Setup Steps

In [0]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "P19A06E_test" #@param {type:"string"}


In [0]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "981234567" #@param {type:"string"}


In [0]:
#@title Run this cell to complete the setup for this Notebook

from IPython import get_ipython
ipython = get_ipython()
  
notebook="BLR_M2_Hackathon" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/Noisy_data.zip")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/studio_data.zip")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/Record_audio.py")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/net_speech_89.pt")
    ipython.magic("sx unzip studio_data.zip")
    ipython.magic("sx unzip Noisy_data.zip")
    ipython.magic("sx pip install torch torchvision")
    ipython.magic("sx pip install librosa")
    print ("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "id" : Id, "file_hash" : file_hash, "notebook" : notebook}

      r = requests.post(url, data = data)
      print("Your submission is successful. Ref:", submission_id)
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if Additional: return Additional      
    else: raise NameError('')
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
  
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


In [0]:
import torch
from torch.autograd import Variable
import numpy as np
import librosa
import os
import warnings
from time import sleep
import sys
warnings.filterwarnings('ignore')

## Pretrained Network for deep features


The following function contains code to load a pre-trained network to produces deep features for the audio sample. This network is trained with delta MFCC features of mono channel 8000 bit rate audio sample.

In [0]:
def get_network():

    net = torch.nn.Sequential()

    saved_net = torch.load("net_speech_89.pt").cpu()

    for index, module in enumerate(saved_net):
        net.add_module("layer"+str(index),module)
        if (index+1)%17 == 0 :
            break
    return net

In [0]:
get_network()

Sequential(
  (layer0): Linear(in_features=900, out_features=800, bias=True)
  (layer1): ReLU()
  (layer2): Linear(in_features=800, out_features=700, bias=True)
  (layer3): ReLU()
  (layer4): Linear(in_features=700, out_features=600, bias=True)
  (layer5): ReLU()
  (layer6): Linear(in_features=600, out_features=500, bias=True)
  (layer7): ReLU()
  (layer8): Linear(in_features=500, out_features=400, bias=True)
  (layer9): ReLU()
  (layer10): Linear(in_features=400, out_features=300, bias=True)
  (layer11): ReLU()
  (layer12): Linear(in_features=300, out_features=200, bias=True)
  (layer13): ReLU()
  (layer14): Linear(in_features=200, out_features=100, bias=True)
  (layer15): ReLU()
  (layer16): Linear(in_features=100, out_features=50, bias=True)
)

##Obtaining Features from Audio samples
Generate features from a audio sample of '.wav' format
* Generate Delta MFCC features of order 1 and 2 
* Passes them through the above mentioned deep neural net
* the obtained deep features are returned

Parameters: Filepath (path of audio sample),
                       sr (sampling rate, all the samples provided are of 8000 bitrate)
         
  Caution: Do not change the default parameters

In [0]:
def get_features(filepath, sr=8000, n_mfcc=30, n_mels=128, frames = 15):
    
    
    y, sr = librosa.load(filepath, sr=sr)
    D = np.abs(librosa.stft(y))**2
    S = librosa.feature.melspectrogram(S=D)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    log_S = librosa.power_to_db(S,ref=np.max)
    features = librosa.feature.mfcc(S=log_S, n_mfcc=n_mfcc)
    if features.shape[1] < frames :
        features = np.hstack((features, np.zeros((n_mfcc, frames - features.shape[1]))))
    elif features.shape[1] > frames:
        features = features[:, :frames]
    # Find 1st order delta_mfcc
    delta1_mfcc = librosa.feature.delta(features, order=1)

    # Find 2nd order delta_mfcc
    delta2_mfcc = librosa.feature.delta(features, order=2)
    features = np.hstack((delta1_mfcc.flatten(), delta2_mfcc.flatten()))
    features = features.flatten()[np.newaxis, :]
    features = Variable(torch.from_numpy(features)).float()
    deep_net = get_network()
    deep_features = deep_net(features)
    #print(features.shape)
    #print(audio_file)
    #features.flatten()[np.newaxis, :]
    return deep_features.data.numpy().flatten()

## All the voice sample needed for training are present across the folders "Noisy_data" and "studio_data"

In [0]:
%ls

M2_Hackathon.ipynb  Noisy_data/       Record_audio.py    studio_data.zip
__MACOSX/           Noisy_data.zip    Record_audio.py.1  studio_data.zip.1
net_speech_89.pt    Noisy_data.zip.1  Record_audio.py.2  studio_data.zip.2
net_speech_89.pt.1  Noisy_data.zip.2  sample_data/
net_speech_89.pt.2  __pycache__/      studio_data/


##Stage 1: 
#### Max. Marks: 5
#### Complete the code in the load_data function

* The function should take path of the folder containing audio samples as input
* It should return features of all the audio samples present in the specified folder into single array (list of lists or 2-d numpy array) and their respective labels should be returned too

In [0]:
def load_data(folder_path):
  ##YOUR CODE HERE
  return features, labels

In [0]:
%ls

M2_Hackathon.ipynb  Noisy_data/       Record_audio.py    studio_data.zip
__MACOSX/           Noisy_data.zip    Record_audio.py.1  studio_data.zip.1
net_speech_89.pt    Noisy_data.zip.1  Record_audio.py.2  studio_data.zip.2
net_speech_89.pt.1  Noisy_data.zip.2  sample_data/
net_speech_89.pt.2  __pycache__/      studio_data/


####load data and labels from studio_data folder

In [0]:
studio_recorded_features, studio_recorded_labels = load_data('studio_data')

##Stage 2:

####Max. Marks:15

#### Train a classifier on the features obtained above

In [0]:
##YOUR CODE HERE

####Save your model

Hint:
* Incase if you are using scikit learn model for training, you can use joblib package to save the model.
* Manually implemented models as a function or class can be saved using pickle 

#### Download your trained model using the code below
* given the path of model file the following code downloads it through the browser

In [0]:
from google.colab import files
files.download('<model_file_path>')


##Stage 3: (Application Deployment)

#### Max. Marks: 5M
#### Now deploy the model trained on studio_data in the sever to order food correctly. 
#### Deployment instruction are given the Hackathon documentation
#### After deploying and checking the application come back here to train on Noisy_data to generalise better in real situations

#### load data and labels from Noisy_data folder

In [0]:
noisy_data, noisy_data_labels = load_data('Noisy_data')

## Stage 4:
####Max. Marks:5
#### Train a classifier on the features obtained from both the Noisy_data and Studio_data

In [0]:
##YOUR CODE HERE

(11839, 50)

#### Download your trained model using the code below
* given the path of model file the following code downloads it through the browser

In [0]:
from google.colab import files
files.download('<model_file_path>')

#### Now deploy the model trained above in the sever to order food correctly. 
#### Deployment instruction are given the Hackathon documentation
#### After deploying and checking the application, record your teams data from the web application provided in the Hackathon document

In [0]:
!mkdir teamdata

Replace <YOUR_GROUP_ID> with your group id given in the lab

In [0]:
!wget -r -A .wav https://aiml-sandbox1.talentsprint.com/audio_recorder/<YOUR_GROUP_ID>/team_data/ -nH --cut-dirs=100  -P ./team_data

In [0]:
%ls

sample_data/  team_data/


In [0]:
!unzip <zip_file_name>

##Stage 5:
#### Max. Marks: 10

## Enhance the model trained with both the noisy data and studio_data to your team's voice samples

In [0]:
##YOUR CODE HERE

### Now deploy the model trained above in the sever to order food correctly. 
### Deployment instruction are given the Hackathon documentation

### Please answer the questions below to complete the experiment:

In [0]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good and Challenging me" #@param ["Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging me", "Was Tough, but I did it", "Too Difficult for me"]


In [0]:
#@title If it was very easy, what more you would have liked to have been added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "test" #@param {type:"string"}

In [0]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["Yes", "No"]

In [0]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")